In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import seaborn as sns
import math
from IPython.display import Markdown, display
import importlib
import json
from tabulate import tabulate
import colorsys
from scipy.ndimage.filters import gaussian_filter
import sklearn
import matplotlib
import xgboost as xgb
import random
from tqdm import tqdm as tqdm


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import src.cropnet as cropnet


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def printmd(string):
    display(Markdown(string))
    
import tensorflow as tf
tf.version.VERSION

/tmp/ipykernel_13818/3764464123.py:12: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter
2024-12-05 10:46:45.144556: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-05 10:46:45.171426: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-05 10:46:45.654849: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


'2.16.1'

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [3]:
path = "data/full_data/processed/processed_merged.feather"
df = pd.read_feather(path)
f = open(path.replace(".feather", ".json"), "r")
correlation_filters = json.load(f)
f.close()
if 'level_0' in df.columns:
    df = df.drop(columns=['level_0'])
df = df.reset_index()
if 'level_0' in df.columns:
    df = df.drop(columns=['level_0'])

df = df[np.isnan(df['yield']) == False]

In [4]:
target = 'yield_1'
predictors = [c for c in correlation_filters['base'] if c not in ['index', 'x', 'y', 'x_int', 'y_int', 'polygon_id', 'grdkod_mar'] and "yield" not in c and "_pid" not in c]


In [5]:

mX = np.mean(df[predictors], axis=0)
sX = np.std(df[predictors], axis=0)
sX[sX == 0] = 1

mY = np.mean(df[target], axis=0)
sY = np.std(df[target], axis=0)

norm_info = {
    'X': {'mean': mX, 'std': sX},
    'y': {'mean': mY, 'std': sY}
}


In [6]:

dataset = []
for pid in tqdm(set( df['matching_pid'])):
    pdf = df[df['matching_pid'] == pid]
    X = ((pdf.loc[df['matching_pid'] == pid, predictors]-mX)/sX).values
    y = ((pdf.loc[df['matching_pid'] == pid, target]-mY)/sY).values.reshape((-1, 1))
    info = {
        'x': pdf.x.values,
        'y': pdf.y.values,
        'pid': pid
    }
    dataset.append((X, y, info))

100%|██████████| 152/152 [00:06<00:00, 23.93it/s]


In [7]:
def split_data(dataset, p_val, p_test, seed=-1):
    
    N = np.sum([X.shape[0] for X, y, info in dataset])
    rem = [data for data in dataset]
    
    data_val = []
    val_size = 0
    while val_size/N < p_val:
        idx = random.choice(range(len(rem)))
        data = rem.pop(idx)
        data_val.append(data)
        val_size += data[0].shape[0]
        
        
    data_test = []
    test_size = 0
    while test_size/N < p_test:
        idx = random.choice(range(len(rem)))
        data = rem.pop(idx)
        data_test.append(data)
        test_size += data[0].shape[0]
        
    return rem, data_val, data_test


In [ ]:
import sklearn
from sklearn import datasets
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb


import concurrent.futures
import multiprocessing

model_list = [
	#LinearRegression(),
	#SVR(),
	#Ridge(),
	#Lasso(),
	#BaggingRegressor(),
	#AdaBoostRegressor(),
	#KNeighborsRegressor(),
	#GradientBoostingRegressor(),
	#SGDRegressor(random_state=0),
	xgb.XGBRegressor(random_state=0),
	#ExtraTreesRegressor(random_state=0),
	#DecisionTreeRegressor(random_state=0),
	#RandomForestRegressor(random_state=0),
    #MLPRegressor(hidden_layer_sizes=(512,128, 64,32,16,4),random_state=0,max_iter=20)
]

for model in model_list:

    print("")
    print(str(model).split("(")[0] + "()")
    model.fit(X_train, y_train)
    y_p = model.predict(X_val)

    err = y_val - y_p
    print("MSE:", np.mean(err**2))
    print("R2:", sklearn.metrics.r2_score(y_val, y_p))
    print("adj_R2:", 1 - (1-model.score(X_val, y_val))*(len(y_val)-1)/(len(y_val)-X_val.shape[1]-1))
    print("CV:", np.std(y_val, ddof=1) / np.mean(y_val))
    #break


In [25]:
score_list = []
for i in range(10):
    data_train, data_val, data_test = split_data(dataset, 0.15, 0.15)

    Xt = np.concatenate([data[0] for data in data_train])
    yt = np.concatenate([data[1] for data in data_train])

    model = xgb.XGBRegressor(random_state=0)
    model.fit(Xt, yt);

    y_tot = np.zeros(0)
    y_pred = np.zeros(0)
    test_err = np.zeros(0)
    test_err_norm = np.zeros(0)
    pids = np.zeros(0)
    for X, y, info in data_test:
        pred = model.predict(X)
        
        yp = pred.reshape(-1)
        yt = y.reshape(-1)
        err = (yt - yp)*norm_info['y']['std']
        
        y_tot = np.concatenate([y_tot, yt])
        y_pred = np.concatenate([y_pred, yp])
        test_err = np.concatenate([test_err, err])
        test_err_norm = np.concatenate([test_err_norm, err - np.nanmean(err)])
        pids = np.concatenate([pids, np.ones(err.shape)*info['pid']])
        
    y_tot = y_tot*norm_info['y']['std'] + norm_info['y']['mean']
    scores_unnorm = [np.sqrt(np.nanmean(test_err**2)), np.nanmean((np.abs(test_err) < 1)[np.isnan(test_err) == False]), np.nanmean(np.abs(test_err/y_tot)), 1-np.sum(test_err**2)/np.sum((y_tot-np.mean(y_tot))**2)]
    scores_norm = [np.sqrt(np.nanmean(test_err_norm**2)), np.nanmean((np.abs(test_err_norm) < 1)[np.isnan(test_err_norm) == False]), np.nanmean(np.abs(test_err_norm/y_tot)), 1-np.sum(test_err_norm**2)/np.sum((y_tot-np.mean(y_tot))**2)]
    result = [scores_unnorm, scores_norm, [list(y_tot), list(y_pred), list(test_err), list(test_err_norm), list(pids)]]
    score_list.append(result)
    print(scores_unnorm, scores_norm)
    
cropnet.save_score_to_json(score_list, "results_final/other_models/xgboost.json")


with open('results_final/other_models/xgboost.json', 'r') as f:
    scores_read = json.load(f)
    print(np.mean(np.array([s[0] for s in scores_read]), axis=0))
    print(np.mean(np.array([s[1] for s in scores_read]), axis=0))

[0.9395790581226872, 0.7311865930396815, 0.09616125279623527, 0.7903349157417794] [0.7933992415003819, 0.8276775394888918, 0.07998416961914041, 0.8504993716303069]
[1.3204075532112147, 0.6390156207511255, 0.12033915457064907, 0.5175783069149744] [0.9350589006909552, 0.7679245853098469, 0.09853498699810839, 0.7580705346545531]
[1.2678699512605494, 0.6776459112339457, 0.1240359732310901, 0.7202425726092636] [0.8580925747328405, 0.7886034791090879, 0.09514198385661442, 0.8718553452680184]
[1.0610103204624588, 0.7442947484190267, 0.12192259548334726, 0.7713260907104742] [0.8812253823639268, 0.8071940351614938, 0.10582137927013092, 0.8422565280770424]
[1.1148143524518725, 0.7010895922920237, 0.11201234589114949, 0.8233369191967976] [0.899938577931958, 0.8019172944485757, 0.09356241634462621, 0.8848758502772173]
[1.026604980161377, 0.7432297339639814, 0.10272723691506362, 0.7670937537740425] [0.8937529014797954, 0.7868429298279643, 0.08985662396363166, 0.8234737352236265]
[0.9661735346528653